<a href="https://colab.research.google.com/github/mandarbagwe7/Hands-On-Large-Language-Models-Practice/blob/main/chapter_03/Looking_inside_LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install transformers==2.5.0

In [3]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

In [ ]:
# Load model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct",
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=True,
)

In [8]:
# Create a pipeline
generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    return_full_text=False,
    max_new_tokens=50,
    do_sample=False,
)

Device set to use cuda
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


In [9]:
prompt = "Write an email apologizing to Sarah for the tragic gardening mishap. Explain how it happened."

output = generator(prompt, use_cache=False)

print(output[0]['generated_text'])

 Mention the steps you're taking to prevent it in the future.

Email:

Subject: Sincere Apologies for the Gardening Mishap

Dear Sarah,

I hope this email finds you well


In [10]:
prompt = "The capital of France is"

# Tokenize the input prompt
input_ids = tokenizer(prompt, return_tensors="pt").input_ids

# Tokenize the input prompt
input_ids = input_ids.to("cuda")

# Get the output of the model before the lm_head
model_output = model.model(input_ids, use_cache=False)

# Get the output of the lm_head
lm_head_output = model.lm_head(model_output[0])

In [11]:
model_output[0].shape

torch.Size([1, 5, 3072])

In [12]:
lm_head_output.shape

torch.Size([1, 5, 32064])

In [13]:
lm_head_output

tensor([[[24.7500, 24.8750, 22.7500,  ..., 19.0000, 19.0000, 19.0000],
         [31.1250, 31.5000, 26.0000,  ..., 26.0000, 26.0000, 26.0000],
         [31.5000, 28.8750, 31.1250,  ..., 26.2500, 26.2500, 26.2500],
         [33.0000, 31.8750, 36.0000,  ..., 27.8750, 27.8750, 27.8750],
         [27.8750, 29.5000, 28.1250,  ..., 20.5000, 20.5000, 20.5000]]],
       device='cuda:0', dtype=torch.bfloat16, grad_fn=<UnsafeViewBackward0>)

In [14]:
lm_head_output[0, -1]

tensor([27.8750, 29.5000, 28.1250,  ..., 20.5000, 20.5000, 20.5000],
       device='cuda:0', dtype=torch.bfloat16, grad_fn=<SelectBackward0>)

In [15]:
token_id = lm_head_output[0, -1].argmax(-1)
tokenizer.decode(token_id)

'Paris'

## Caching

In [ ]:
!pip install --upgrade transformers

In [14]:
from transformers import AutoModelForCausalLM, AutoTokenizer

In [15]:
# Load model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct",
    device_map="cuda",
    torch_dtype="auto",
    # trust_remote_code=True,
)

In [17]:
prompt = "Write a very long email apologizing to Sarah for the tragic gardening mishap. Explain how it happened."

In [18]:
# Tokenize the input prompt
input_ids = tokenizer(prompt, return_tensors="pt").input_ids
input_ids = input_ids.to("cuda")

In [19]:
%%timeit -n 1

# Generate the text
generation_output = model.generate(
input_ids=input_ids,
max_new_tokens=100,
use_cache=True
)

4.81 s ± 284 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [20]:
%%timeit -n 1

# Generate the text
generation_output = model.generate(
input_ids=input_ids,
max_new_tokens=100,
use_cache=False
)

31 s ± 397 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
